# Import packages and Settings

In [1]:
import os

n_core = os.cpu_count()-3
print(n_core)

9


In [2]:
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype

import matplotlib.pyplot as plt

from xgboost.sklearn import XGBRegressor

from catboost import CatBoostRegressor

%matplotlib inline

In [3]:
from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import RFECV
from sklearn.model_selection import ShuffleSplit, cross_val_score, GridSearchCV

from sklearnex import patch_sklearn

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Load Data and Preprocessing

In [4]:
train_data = pd.read_csv('./Data/train.csv')
test_data = pd.read_csv('./Data/test.csv')

feature = 'MSSubClass'
train_data[feature] = train_data[feature].apply(lambda x: str(x))
test_data[feature] = test_data[feature].apply(lambda x: str(x))

assert(not is_numeric_dtype(train_data[feature]))

feature = 'Street'
train_data[feature] = train_data[feature].map({'Pave': 0, 'Grvl':1})
test_data[feature] = test_data[feature].map({'Pave': 0, 'Grvl':1})

assert(is_numeric_dtype(train_data[feature]))

feature = 'Alley'
train_data[feature] = train_data[feature].fillna('None')
test_data[feature] = test_data[feature].fillna('None')

assert(train_data[feature].isna().sum() == 0)

feature = 'MasVnrType'
train_data[feature] = train_data[feature].fillna('None')
test_data[feature] = test_data[feature].fillna('None')

assert(train_data[feature].isna().sum() == 0)

feature = 'MasVnrArea'
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)

assert(train_data[feature].isna().sum() == 0)

quality_map = {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}

feature = 'ExterQual'
train_data[feature] = train_data[feature].map(quality_map)
test_data[feature] = test_data[feature].map(quality_map)

assert(is_numeric_dtype(train_data[feature]))

feature = 'ExterCond'
train_data[feature] = train_data[feature].map(quality_map)
test_data[feature] = test_data[feature].map(quality_map)

assert(is_numeric_dtype(train_data[feature]))

feature = 'BsmtQual'
train_data[feature] = train_data[feature].map(quality_map)
test_data[feature] = test_data[feature].map(quality_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

feature = 'BsmtCond'
train_data[feature] = train_data[feature].map(quality_map)
test_data[feature] = test_data[feature].map(quality_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

exposure_map = {'No':1, 'Mn':2, 'Av':3, 'Gd':4}

feature = 'BsmtExposure'
train_data[feature] = train_data[feature].map(exposure_map)
test_data[feature] = test_data[feature].map(exposure_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

type_map = {'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}

feature = 'BsmtFinType1'
train_data[feature] = train_data[feature].map(type_map)
test_data[feature] = test_data[feature].map(type_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

feature = 'BsmtFinType2'
train_data[feature] = train_data[feature].map(type_map)
test_data[feature] = test_data[feature].map(type_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

feature = 'HeatingQC'
train_data[feature] = train_data[feature].map(quality_map)
test_data[feature] = test_data[feature].map(quality_map)

assert(is_numeric_dtype(train_data[feature]))

feature = 'CentralAir'
train_data[feature] = train_data[feature].map({'N': 0, 'Y':1})
test_data[feature] = test_data[feature].map({'N': 0, 'Y':1})

assert(is_numeric_dtype(train_data[feature]))

feature = 'KitchenQual'
train_data[feature] = train_data[feature].map(quality_map)
test_data[feature] = test_data[feature].map(quality_map)

assert(is_numeric_dtype(train_data[feature]))

function_map = {'Sal':1, 'Sev':2, 'Maj2':3, 'Maj1':4, 'Mod':5, 'Min2':6, 'Min1':7, 'Typ':8}

feature = 'Functional'
train_data[feature] = train_data[feature].map(function_map)
test_data[feature] = test_data[feature].map(function_map)
test_data[feature] = test_data[feature].fillna(8)

assert(is_numeric_dtype(train_data[feature]))

feature = 'FireplaceQu'
train_data[feature] = train_data[feature].map(quality_map)
test_data[feature] = test_data[feature].map(quality_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

feature = 'GarageType'
train_data[feature] = train_data[feature].fillna('None')
test_data[feature] = test_data[feature].fillna('None')

assert(train_data[feature].isna().sum() == 0)

feature = 'GarageYrBlt'
train_data[feature] = train_data[feature].fillna(-1)
test_data[feature] = test_data[feature].fillna(-1)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(train_data[feature].isna().sum() == 0)

exposure_map = {'Unf':1, 'RFn':2, 'Fin':3}

feature = 'GarageFinish'
train_data[feature] = train_data[feature].map(exposure_map)
test_data[feature] = test_data[feature].map(exposure_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

feature = 'GarageQual'
train_data[feature] = train_data[feature].map(quality_map)
test_data[feature] = test_data[feature].map(quality_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

feature = 'GarageCond'
train_data[feature] = train_data[feature].map(quality_map)
test_data[feature] = test_data[feature].map(quality_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

feature = 'PoolQC'
train_data[feature] = train_data[feature].map(quality_map)
test_data[feature] = test_data[feature].map(quality_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

fence_map = {'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4}

feature = 'Fence'
train_data[feature] = train_data[feature].map(fence_map)
test_data[feature] = test_data[feature].map(fence_map)
train_data[feature] = train_data[feature].fillna(0)
test_data[feature] = test_data[feature].fillna(0)
train_data[feature] = train_data[feature].astype(int)
test_data[feature] = test_data[feature].astype(int)

assert(is_numeric_dtype(train_data[feature]))
assert(train_data[feature].isna().sum() == 0)

feature = 'MiscFeature'
train_data[feature] = train_data[feature].fillna('None')
test_data[feature] = test_data[feature].fillna('None')

assert(train_data[feature].isna().sum() == 0)

test_data.loc[test_data['GarageCars'].isnull(), ['GarageType', 'GarageCars', 'GarageArea']] = ['None', 0, 0]
test_data.loc[test_data['BsmtFinSF1'].isnull(), ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']] = [0, 0, 0, 0, 0, 0]
test_data.loc[test_data['BsmtFullBath'].isnull(), ['BsmtFullBath', 'BsmtHalfBath']] = [0, 0]

assert(test_data['GarageCars'].isna().sum() == 0)
assert(test_data['BsmtFinSF1'].isna().sum() == 0)
assert(test_data['BsmtFullBath'].isna().sum() == 0)

columns_float = train_data.select_dtypes(include=['float64']).columns
train_data[columns_float] = train_data[columns_float].astype('float32')
test_data[columns_float] = test_data[columns_float].astype('float32')

label = 'SalePrice'

id = test_data['Id']

In [5]:
models = [
    LinearRegression(),
    Lasso(random_state=0),
    Ridge(random_state=0),
    DecisionTreeRegressor(random_state=0),
    RandomForestRegressor(random_state=0, oob_score=True),
    GradientBoostingRegressor(random_state=0),
    XGBRegressor(random_state=0),
    CatBoostRegressor(random_state=0, verbose=0, allow_writing_files=False)
]

# Functions

In [6]:
def prepareDataDrop(columns_to_drop, label, train_data, test_data, islog):
    X_train = train_data.drop([label] + columns_to_drop, axis=1)
    Y_train = np.log(train_data[label]) if islog else train_data[label]
    X_test = test_data.drop(columns_to_drop, axis=1)
    return X_train, Y_train, X_test

In [7]:
def prepareDataSelect(columns_to_select, label, train_data, test_data, islog):
    X_train = train_data[columns_to_select]
    Y_train = np.log(train_data[label]) if islog else train_data[label]
    X_test = test_data[columns_to_select]
    return X_train, Y_train, X_test

In [8]:
def preprocess(X_train, X_test):
    columns_num = X_train.select_dtypes(include='number').columns
    columns_cat = X_train.select_dtypes(include='O').columns

    X_train[columns_num] = X_train[columns_num].fillna(X_train[columns_num].median())
    X_train[columns_cat] = X_train[columns_cat].fillna(X_train[columns_cat].mode().loc[0])

    X_test[columns_num] = X_test[columns_num].fillna(X_train[columns_num].median())
    X_test[columns_cat] = X_test[columns_cat].fillna(X_train[columns_cat].mode().loc[0])

    encoder_onehot = OneHotEncoder(dtype=int, sparse_output=False, handle_unknown='ignore')

    train_data_onehot = encoder_onehot.fit_transform(X_train[columns_cat])
    train_feature_name_onehot = encoder_onehot.get_feature_names_out()
    train_data_onehot = pd.DataFrame(train_data_onehot, columns=train_feature_name_onehot)

    test_data_onehot = encoder_onehot.transform(X_test[columns_cat])
    test_data_onehot = pd.DataFrame(test_data_onehot, columns=train_feature_name_onehot)

    X_train = pd.concat([X_train[columns_num], train_data_onehot], axis=1)
    X_test = pd.concat([X_test[columns_num], test_data_onehot], axis=1)

    return X_train, X_test

In [9]:
def evaluation(models, X_train, Y_train, scoring, n_core):
    model_names = [model.__class__.__name__ for model in models]
    model_scores = []

    cv = ShuffleSplit(random_state=0)

    for model in models:
        model = clone(model)
        cv_scores = cross_val_score(model, X_train, Y_train, cv=cv, scoring=scoring, n_jobs=n_core)
        model_scores.append(-cv_scores.mean())

    cv_results = pd.DataFrame(columns=['name', 'score_mean'])
    cv_results['name'] = model_names
    cv_results['score_mean'] = model_scores

    return cv_results

In [10]:
def tuneParameters(models, tuning_parameters, X_train, Y_train, scoring, n_core):
    model_names = [model.__class__.__name__ for model in models]
    model_scores = []
    model_params = []

    cv = ShuffleSplit(random_state=0)

    for model, params in zip(models, tuning_parameters.items()):
        model = clone(model)
        tune_model = GridSearchCV(model, param_grid=params[1], scoring=scoring, cv=cv, n_jobs=n_core)
        tune_model.fit(X_train, Y_train)
        model_scores.append(-tune_model.best_score_)
        model_params.append(tune_model.best_params_)
    
    tuning_results = pd.DataFrame(columns=['name', 'best_score', 'best_params'])
    tuning_results['name'] = model_names
    tuning_results['best_score'] = model_scores
    tuning_results['best_params'] = model_params

    return tuning_results

In [11]:
def predict(model, X_train, Y_train, X_test, islog, id, label, file_name = 'result.csv'):
    model = clone(model)
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)

    result = pd.DataFrame({
        id.name: id,
        label: np.exp(predictions) if islog else predictions
    })
    result.to_csv('./Result/' + file_name, index=False)

In [84]:
train_data.columns[train_data.isna().any()].tolist()

['LotFrontage', 'Electrical']

In [85]:
test_data.columns[test_data.isna().any()].tolist()

['MSZoning',
 'LotFrontage',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'KitchenQual',
 'SaleType']

# Base Model

In [86]:
columns_to_drop = ['Id'] + \
    train_data.select_dtypes(include='O').columns.to_list() + \
    train_data.columns[train_data.isna().any()].tolist()
X_train, Y_train, X_test = prepareDataDrop(columns_to_drop, label, train_data, test_data, False)
print(X_train.columns)

Index(['LotArea', 'Street', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'PoolQC', 'Fence', 'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')


In [88]:
cv_results = evaluation(models, X_train, Y_train, 'neg_root_mean_squared_log_error', n_core)
print(cv_results)

                        name  score_mean
0           LinearRegression    0.175144
1                      Lasso    0.175102
2                      Ridge    0.174548
3      DecisionTreeRegressor    0.206714
4      RandomForestRegressor    0.139624
5  GradientBoostingRegressor    0.126800
6               XGBRegressor    0.138030
7          CatBoostRegressor    0.124243


In [13]:
model = DecisionTreeRegressor(random_state=0)
predict(model, X_train, Y_train, X_test, False, id, label, file_name = 'result_basic.csv')

# Improvement

In [64]:
columns_to_drop = ['Id']
X_train, Y_train, X_test = prepareDataDrop(columns_to_drop, label, train_data, test_data, False)
print(X_train.columns)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [65]:
X_train, X_test = preprocess(X_train, X_test)
print(X_train.shape, X_test.shape)

(1460, 245) (1459, 245)


In [68]:
cv_results = evaluation(models, X_train, Y_train, 'neg_root_mean_squared_log_error', n_core)
print(cv_results)

                        name  score_mean
0           LinearRegression         NaN
1                      Lasso         NaN
2                      Ridge         NaN
3      DecisionTreeRegressor    0.204164
4      RandomForestRegressor    0.139421
5  GradientBoostingRegressor    0.122607
6               XGBRegressor    0.136823
7          CatBoostRegressor    0.114957


In [22]:
tuning_parameters = {
        'RandomForestRegressor': {
            'n_estimators': [50, 100, 300],
            'min_samples_split': [2, 4, 6, 8, 10]
        }
    }

model = RandomForestRegressor(random_state=0, oob_score=True)

tuneParameters([model], tuning_parameters, X_train, Y_train, 'neg_root_mean_squared_log_error', n_core)

,name,best_score,best_params
0,RandomForestRegressor,-0.138408,"{'min_samples_split': 2, 'n_estimators': 300}"


In [30]:
model = RandomForestRegressor(n_estimators=300, min_samples_split=10, random_state=0, oob_score=True)
predict(model, X_train, Y_train, X_test, False, id, label)

## Take Log of SalePrice

In [89]:
columns_to_drop = ['Id']
X_train, Y_train, X_test = prepareDataDrop(columns_to_drop, label, train_data, test_data, True)
print(X_train.columns)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [90]:
X_train, X_test = preprocess(X_train, X_test)
print(X_train.shape, X_test.shape)

(1460, 245) (1459, 245)


In [27]:
cv_results = evaluation(models, X_train, Y_train, 'neg_root_mean_squared_error', n_core)
print(cv_results)

                        name  score_mean
0           LinearRegression    0.140600
1                      Lasso    0.186531
2                      Ridge    0.133502
3      DecisionTreeRegressor    0.198165
4      RandomForestRegressor    0.136668
5  GradientBoostingRegressor    0.122738
6               XGBRegressor    0.141680
7          CatBoostRegressor    0.113981


In [29]:
tuning_parameters = {
        'RandomForestRegressor': {
            'n_estimators': [50, 100, 300],
            'min_samples_split': [2, 4, 6, 8, 10]
        }
    }

model = RandomForestRegressor(random_state=0, oob_score=True)

tuneParameters([model], tuning_parameters, X_train, Y_train, 'neg_root_mean_squared_error', n_core)

,name,best_score,best_params
0,RandomForestRegressor,-0.136071,"{'min_samples_split': 2, 'n_estimators': 300}"


In [21]:
model = RandomForestRegressor(n_estimators=300, min_samples_split=10, random_state=0, oob_score=True)
predict(model, X_train, Y_train, X_test, True, id, label)

## Remove Outliers

In [70]:
train_data_new = train_data.drop(train_data[(train_data['OverallQual']==4) & (train_data['SalePrice']>200000)].index)
train_data_new = train_data_new.drop(train_data_new[(train_data_new['GrLivArea']>4000) & (train_data_new['SalePrice']<300000)].index)

In [71]:
columns_to_drop = ['Id']
X_train, Y_train, X_test = prepareDataDrop(columns_to_drop, label, train_data_new, test_data, True)
print(X_train.columns)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [72]:
X_train, X_test = preprocess(X_train, X_test)
print(X_train.shape, X_test.shape)

(1457, 244) (1459, 244)


In [23]:
cv_results = evaluation(models, X_train, Y_train, 'neg_root_mean_squared_error', n_core)
print(cv_results)

                        name  score_mean
0           LinearRegression    0.122964
1                      Lasso    0.164459
2                      Ridge    0.113877
3      DecisionTreeRegressor    0.195040
4      RandomForestRegressor    0.128458
5  GradientBoostingRegressor    0.117576
6               XGBRegressor    0.126859
7          CatBoostRegressor    0.107263


In [24]:
tuning_parameters = {
        'RandomForestRegressor': {
            'n_estimators': [50, 100, 300],
            'min_samples_split': [2, 4, 6, 8, 10]
        }
    }

model = RandomForestRegressor(random_state=0, oob_score=True)

tuneParameters([model], tuning_parameters, X_train, Y_train, 'neg_root_mean_squared_error', n_core)

,name,best_score,best_params
0,RandomForestRegressor,0.127632,"{'min_samples_split': 4, 'n_estimators': 300}"


In [25]:
model = RandomForestRegressor(n_estimators=300, min_samples_split=4, random_state=0, oob_score=True)
predict(model, X_train, Y_train, X_test, True, id, label)

## Top 10 Features

In [91]:
train_data_new = train_data.drop(train_data[(train_data['OverallQual']==4) & (train_data['SalePrice']>200000)].index)
train_data_new = train_data_new.drop(train_data_new[(train_data_new['GrLivArea']>4000) & (train_data_new['SalePrice']<300000)].index)

columns_to_select = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath']
X_train, Y_train, X_test = prepareDataSelect(columns_to_select, label, train_data_new, test_data, True)
print(X_train.columns)

Index(['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath'], dtype='object')


In [92]:
X_train, X_test = preprocess(X_train, X_test)
print(X_train.shape, X_test.shape)

(1457, 5) (1459, 5)


In [30]:
cv_results = evaluation(models, X_train, Y_train, 'neg_root_mean_squared_error', n_core)
print(cv_results)

                        name  score_mean
0           LinearRegression    0.160072
1                      Lasso    0.223392
2                      Ridge    0.160073
3      DecisionTreeRegressor    0.225016
4      RandomForestRegressor    0.165167
5  GradientBoostingRegressor    0.155705
6               XGBRegressor    0.171244
7          CatBoostRegressor    0.157339


In [31]:
tuning_parameters = {
        'RandomForestRegressor': {
            'n_estimators': [50, 100, 300],
            'min_samples_split': [2, 4, 6, 8, 10]
        }
    }

model = RandomForestRegressor(random_state=0, oob_score=True)

tuneParameters([model], tuning_parameters, X_train, Y_train, 'neg_root_mean_squared_error', n_core)

,name,best_score,best_params
0,RandomForestRegressor,0.160232,"{'min_samples_split': 10, 'n_estimators': 300}"


## Drop Duplicated Features

In [93]:
train_data_new = train_data.drop(train_data[(train_data['OverallQual']==4) & (train_data['SalePrice']>200000)].index)
train_data_new = train_data_new.drop(train_data_new[(train_data_new['GrLivArea']>4000) & (train_data_new['SalePrice']<300000)].index)

columns_to_drop = ['Id', 'GarageArea', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtCond', 'BsmtFinSF1', 'BsmtFinSF2']
X_train, Y_train, X_test = prepareDataDrop(columns_to_drop, label, train_data_new, test_data, True)
print(X_train.columns)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageCars', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscFeatur

In [94]:
X_train, X_test = preprocess(X_train, X_test)
print(X_train.shape, X_test.shape)

(1457, 236) (1459, 236)


In [34]:
cv_results = evaluation(models, X_train, Y_train, 'neg_root_mean_squared_error', n_core)
print(cv_results)

                        name  score_mean
0           LinearRegression    0.122518
1                      Lasso    0.172266
2                      Ridge    0.114146
3      DecisionTreeRegressor    0.196435
4      RandomForestRegressor    0.129300
5  GradientBoostingRegressor    0.116507
6               XGBRegressor    0.127121
7          CatBoostRegressor    0.109326


In [35]:
tuning_parameters = {
        'RandomForestRegressor': {
            'n_estimators': [50, 100, 300],
            'min_samples_split': [2, 4, 6, 8, 10]
        }
    }

model = RandomForestRegressor(random_state=0, oob_score=True)

tuneParameters([model], tuning_parameters, X_train, Y_train, 'neg_root_mean_squared_error', n_core)

,name,best_score,best_params
0,RandomForestRegressor,0.128318,"{'min_samples_split': 4, 'n_estimators': 300}"


## Take Log of Top 10 Features

In [95]:
train_data_new = train_data.drop(train_data[(train_data['OverallQual']==4) & (train_data['SalePrice']>200000)].index)
train_data_new = train_data_new.drop(train_data_new[(train_data_new['GrLivArea']>4000) & (train_data_new['SalePrice']<300000)].index)

train_data_new['GrLivArea'] = np.log1p(train_data_new['GrLivArea'])
train_data_new['TotalBsmtSF'] = np.log1p(train_data_new['TotalBsmtSF'])

test_data_new = test_data.copy()

test_data_new['GrLivArea'] = np.log1p(test_data_new['GrLivArea'])
test_data_new['TotalBsmtSF'] = np.log1p(test_data_new['TotalBsmtSF'])

In [96]:
columns_to_select = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath']
X_train, Y_train, X_test = prepareDataSelect(columns_to_select, label, train_data_new, test_data_new, True)
print(X_train.columns)

Index(['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath'], dtype='object')


In [97]:
X_train, X_test = preprocess(X_train, X_test)
print(X_train.shape, X_test.shape)

(1457, 5) (1459, 5)


In [39]:
cv_results = evaluation(models, X_train, Y_train, 'neg_root_mean_squared_error', n_core)
print(cv_results)

                        name  score_mean
0           LinearRegression    0.170753
1                      Lasso    0.386474
2                      Ridge    0.170686
3      DecisionTreeRegressor    0.223832
4      RandomForestRegressor    0.165115
5  GradientBoostingRegressor    0.155808
6               XGBRegressor    0.171244
7          CatBoostRegressor    0.156945


In [40]:
tuning_parameters = {
        'RandomForestRegressor': {
            'n_estimators': [50, 100, 300],
            'min_samples_split': [2, 4, 6, 8, 10]
        }
    }

model = RandomForestRegressor(random_state=0, oob_score=True)

tuneParameters([model], tuning_parameters, X_train, Y_train, 'neg_root_mean_squared_error', n_core)

,name,best_score,best_params
0,RandomForestRegressor,0.160311,"{'min_samples_split': 10, 'n_estimators': 300}"


##  Drop Duplicated Features and Take Log

In [12]:
train_data_new = train_data.drop(train_data[(train_data['OverallQual']==4) & (train_data['SalePrice']>200000)].index)
train_data_new = train_data_new.drop(train_data_new[(train_data_new['GrLivArea']>4000) & (train_data_new['SalePrice']<300000)].index)

skews = train_data_new.select_dtypes(include='number').skew()
columns_skew = skews[skews > 0.5].index

print(columns_skew)

Index(['LotFrontage', 'LotArea', 'Street', 'OverallCond', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'HalfBath',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'PoolQC', 'Fence', 'MiscVal', 'SalePrice'],
      dtype='object')


In [14]:
columns_to_log = ['LotFrontage', 'LotArea', 'MasVnrArea', 
                  'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                  'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
                  'LowQualFinSF', 'GrLivArea', 'WoodDeckSF', 
                  'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 
                  'ScreenPorch', 'PoolArea', 'MiscVal']

train_data_new[columns_to_log] = np.log1p(train_data_new[columns_to_log])

test_data_new = test_data.copy()
test_data_new[columns_to_log] = np.log1p(test_data_new[columns_to_log])

In [15]:
columns_to_drop = ['Id', 'GarageArea', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtCond', 'BsmtFinSF1', 'BsmtFinSF2']
X_train, Y_train, X_test = prepareDataDrop(columns_to_drop, label, train_data_new, test_data_new, True)
print(X_train.columns)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageCars', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscFeatur

In [16]:
X_train, X_test = preprocess(X_train, X_test)
print(X_train.shape, X_test.shape)

(1457, 236) (1459, 236)


In [17]:
cv_results = evaluation(models, X_train, Y_train, 'neg_root_mean_squared_error', n_core)
print(cv_results)

                        name    score_mean
0           LinearRegression  2.267725e+06
1                      Lasso  2.916569e-01
2                      Ridge  1.217598e-01
3      DecisionTreeRegressor  1.964215e-01
4      RandomForestRegressor  1.294068e-01
5  GradientBoostingRegressor  1.164239e-01
6               XGBRegressor  1.271215e-01
7          CatBoostRegressor  1.093189e-01


In [18]:
tuning_parameters = {
        'RandomForestRegressor': {
            'n_estimators': [50, 100, 300],
            'min_samples_split': [2, 4, 6, 8, 10]
        }
    }

model = RandomForestRegressor(random_state=0, oob_score=True)

tuneParameters([model], tuning_parameters, X_train, Y_train, 'neg_root_mean_squared_error', n_core)

,name,best_score,best_params
0,RandomForestRegressor,0.128372,"{'min_samples_split': 4, 'n_estimators': 300}"


## New Features

In [12]:
train_data_new, test_data_new = train_data.copy(), test_data.copy()

train_data_new['TotalSF'] = train_data_new[['TotalBsmtSF', '1stFlrSF', '2ndFlrSF']].sum(axis=1)
test_data_new['TotalSF'] = test_data_new[['TotalBsmtSF', '1stFlrSF', '2ndFlrSF']].sum(axis=1)

train_data_new['TotalFinSF'] = train_data_new[['BsmtFinSF1', 'BsmtFinSF2', '1stFlrSF', '2ndFlrSF']].sum(axis=1)
test_data_new['TotalFinSF'] = test_data_new[['BsmtFinSF1', 'BsmtFinSF2', '1stFlrSF', '2ndFlrSF']].sum(axis=1)

train_data_new['YearSinceBuilt'] = train_data_new['YrSold'] - train_data_new['YearBuilt']
test_data_new['YearSinceBuilt'] = test_data_new['YrSold'] - test_data_new['YearBuilt']

train_data_new['YearSinceRemod'] = train_data_new['YrSold'] - train_data_new['YearRemodAdd']
test_data_new['YearSinceRemod'] = test_data_new['YrSold'] - test_data_new['YearRemodAdd']

train_data_new['TotalBath'] = train_data_new[['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath']].sum(axis=1)
test_data_new['TotalBath'] = test_data_new[['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath']].sum(axis=1)

train_data_new['TotalPorch'] = train_data_new[['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']].sum(axis=1)
test_data_new['TotalPorch'] = test_data_new[['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']].sum(axis=1)

train_data_new['HasPool'] = train_data_new['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
test_data_new['HasPool'] = test_data_new['PoolArea'].apply(lambda x: 1 if x > 0 else 0)

train_data_new['Has2ndFloor'] = train_data_new['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
test_data_new['Has2ndFloor'] = test_data_new['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)

train_data_new['HasBsmt'] = train_data_new['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
test_data_new['HasBsmt'] = test_data_new['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)

train_data_new['HasFireplace'] = train_data_new['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)
test_data_new['HasFireplace'] = test_data_new['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

train_data_new['HasGarage'] = train_data_new['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
test_data_new['HasGarage'] = test_data_new['GarageArea'].apply(lambda x: 1 if x > 0 else 0)

train_data_new['HasFence'] = train_data_new['Fence'].apply(lambda x: 1 if x > 0 else 0)
test_data_new['HasFence'] = test_data_new['Fence'].apply(lambda x: 1 if x > 0 else 0)

train_data_new['HasPorch'] = train_data_new['TotalPorch'].apply(lambda x: 1 if x > 0 else 0)
test_data_new['HasPorch'] = test_data_new['TotalPorch'].apply(lambda x: 1 if x > 0 else 0)

In [13]:
columns_to_drop = ['Id', 'GarageArea', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtCond', 'BsmtFinSF1', 'BsmtFinSF2']
X_train, Y_train, X_test = prepareDataDrop(columns_to_drop, label, train_data_new, test_data_new, True)

columns_candidate = X_train.columns
print(X_train.columns)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageCars', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscFeatur

In [14]:
X_train, X_test = preprocess(X_train, X_test)
print(X_train.shape, X_test.shape)

(1460, 250) (1459, 250)


In [15]:
cv_results = evaluation(models, X_train, Y_train, 'neg_root_mean_squared_error', n_core)
print(cv_results)

                        name  score_mean
0           LinearRegression    4.477229
1                      Lasso    0.192534
2                      Ridge    0.132930
3      DecisionTreeRegressor    0.208323
4      RandomForestRegressor    0.136881
5  GradientBoostingRegressor    0.126195
6               XGBRegressor    0.136785
7          CatBoostRegressor    0.114024


In [16]:
cv = ShuffleSplit(random_state=0)
model = RandomForestRegressor(random_state=0, oob_score=True)
selector = RFECV(estimator=model, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=n_core)
selector.fit(X_train, Y_train)

for feature, rank in zip(columns_candidate, selector.ranking_):
    print(feature, rank)

MSSubClass 1
MSZoning 1
LotFrontage 145
LotArea 1
Street 1
Alley 1
LotShape 1
LandContour 1
Utilities 1
LotConfig 1
LandSlope 1
Neighborhood 1
Condition1 1
Condition2 19
BldgType 1
HouseStyle 1
OverallQual 1
OverallCond 1
YearBuilt 1
YearRemodAdd 1
RoofStyle 97
RoofMatl 1
Exterior1st 4
Exterior2nd 56
MasVnrType 1
MasVnrArea 7
ExterQual 1
ExterCond 3
Foundation 1
BsmtQual 1
BsmtExposure 1
BsmtFinType1 1
BsmtFinType2 1
BsmtUnfSF 1
TotalBsmtSF 1
Heating 1
HeatingQC 1
CentralAir 86
Electrical 1
1stFlrSF 135
2ndFlrSF 109
LowQualFinSF 1
GrLivArea 68
BsmtFullBath 1
BsmtHalfBath 1
FullBath 1
HalfBath 1
BedroomAbvGr 1
KitchenAbvGr 1
KitchenQual 1
TotRmsAbvGrd 1
Functional 92
Fireplaces 69
FireplaceQu 121
GarageType 11
GarageCars 26
PavedDrive 43
WoodDeckSF 45
OpenPorchSF 103
EnclosedPorch 66
3SsnPorch 165
ScreenPorch 138
PoolArea 14
PoolQC 8
Fence 148
MiscFeature 146
MiscVal 49
MoSold 1
YrSold 75
SaleType 128
SaleCondition 64
TotalSF 141
TotalFinSF 93
YearSinceBuilt 1
YearSinceRemod 101
TotalBa